# Magnetic Contol program
**`author`**: `Kuan-Chia Chiu`  
**`update`**: `2018-12-12`  
**`package installed`**: `pyvisa`, `serial`, `ipywidgets`(*optional*), `plotly`(*optional*)


In [1]:
import visa
import time
import serial
import matplotlib.pyplot as plt


rm = visa.ResourceManager()
res = rm.list_resources()
vna_dict =[]
for x in range(0, len(res)):
    vna_dict.append(res[x])
    print("{}. ".format(x) + res[x] + ", \n")
print("\n")


0. ASRL4::INSTR, 

1. ASRL5::INSTR, 

2. ASRL6::INSTR, 

3. GPIB0::6::INSTR, 





In [2]:
KEPCO_address = "GPIB0::6::INSTR"
LakeShore_address = "COM6"

In [3]:
ser = serial.Serial(LakeShore_address, 57600,bytesize=7, parity=serial.PARITY_ODD, stopbits=1)  
# Lake Shore 425

## Current to Field curves

In [4]:
sweep_current = 10
sweep_pts = 101
KEPCO = rm.open_resource(KEPCO_address)

KEPCO.timeout=10000
KEPCO.write("FUNC:MODE CURR")
KEPCO.write("volt 0.000;curr 0.000")
KEPCO.write("OUTPUT ON")
Field_data = []
Current_data = []
for i in range(sweep_pts):
    KEPCO.write("volt  20.000;curr {:.4f}".format(i*sweep_current/(sweep_pts-1)))
    time.sleep(1)
    Current = str(i*sweep_current/(sweep_pts-1))
    Current_data.append(Current)
    ser.write(b'RDGFIELD?\n')
    Field = ser.readline()
    Field_data.append(Field.decode('utf8'))
    time.sleep(0.1)

KEPCO.write("OUTPUT OFF")
KEPCO.write("volt 0.000;curr 0.000")
#ser.close()


(23, <StatusCode.success: 0>)

## Save Data

In [6]:
import os
import re
data_filename = './20181213_10A_delay_1s.txt'
# Add datetime automatically and append with the file name arbitaryly.
with open(data_filename, 'w') as f:
    f.write('Cuurent\s(A)\tField\s(G)\n')
    for j in range(sweep_pts):
        f.write('{}\t{}'.format(Current_data[j],Field_data[j]))
f.close()

## Plot data

In [7]:
import plotly as py
from plotly.graph_objs import Scatter, Layout
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
trace = go.Scatter(x=Current_data, y=Field_data, name='Original data')
iplot([trace], filename='test')

## Set the name of data

In [29]:
import datetime as date
##### Set Parameter #####
sweep_current = 30
delay = 1
freq = 9000000000
meas_para = "S21"
#########################

year = date.datetime.now().year
day = date.datetime.now().day
month = date.datetime.now().month
data_str = "{}{}{}_{}_{}A_delay{}s_{:.0f}GHz.txt".format(year,month,day,meas_para,sweep_current,delay,freq/1000000000)
print(data_str)

20181213_S21_30A_delay1s_9GHz.txt
